In [ ]:
import pickle
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import pandas as pd
import seaborn as sns
from AFMReader.topostats import load_topostats
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

from topostats.damage.damage import (Defect, DefectGap, OrderedDefectGapList,
                                     calculate_indirect_defect_gaps,
                                     get_defects_and_gaps_from_bool_array)
from topostats.io import LoadScans
from topostats.measure.curvature import (
    discrete_angle_difference_per_nm_circular, total_turn_in_region_radians)
from topostats.tracing.splining import resample_points_regular_interval
from topostats.unet_masking import (
    make_bounding_box_square, pad_bounding_box_cutting_off_at_image_bounds)

In [ ]:
# Get the data directories set up
dir_base = Path(
    "/Volumes/shared-1/pyne_group/Shared/AFM_Data/dna_damage/Cs137_irradiations/20260204-analysis-getting-back-into-the-project/"
)
assert dir_base.exists()
dir_data = dir_base / "output"
assert dir_data.exists()
dir_processed = dir_base / "analysis_results"
dir_processed.mkdir(exist_ok=True)
assert dir_processed.exists()

# Load the data, lazily since the files are large?
topo_files = list(dir_data.glob("*/**/*.topostats"))
print(f"found {len(topo_files)} topo files")

# Load the corresponding statistics csv file
csv_grain_stats = dir_data / "grain_statistics.csv"
assert csv_grain_stats.exists(), f"could not find grain stats csv at {csv_grain_stats}"
df_grain_stats = pd.read_csv(csv_grain_stats)
print(f"grain stats columns: {df_grain_stats.columns}")

# convert some columns to nanometres
df_grain_stats["total_contour_length"] /= 1e-9

In [ ]:
# plot contour length distributions
sns.stripplot(data=df_grain_stats, x="basename", y="total_contour_length", s=2)
sns.violinplot(data=df_grain_stats, x="basename", y="total_contour_length", inner=None)
plt.xticks(rotation=90)
plt.title("Contour length distributions")
plt.show()